In [28]:
import torch
from torch import nn
import tools
import pandas
import data_process


In [29]:
class lstm(nn.Module):
    def __init__(self,num_embeddings,embedding_dim,hidden_size,device,num_layers=1,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(num_embeddings,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_size,num_layers,bidirectional=True)
        self.ff = nn.Linear(2*hidden_size,2)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device
    def forward(self,X):
        h0 = torch.randn(self.num_layers*2,X.shape[0],self.hidden_size).to(self.device)
        c0 = torch.randn(self.num_layers*2,X.shape[0],self.hidden_size).to(self.device)
        X = self.embedding(X)
        X = X.transpose(0,1)
        output,_ = self.lstm(X,(h0,c0))
        return self.ff(output[-1])
        
        

In [10]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

In [30]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, device, num_layers=1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, dropout=0.5, batch_first=True)
        self.ff = nn.Linear(2 * hidden_size, 2)  # 双向LSTM的输出需要乘以2
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

    def forward(self, X):
        h0 = torch.zeros(self.num_layers * 2, X.shape[0], self.hidden_size).to(self.device)  # 双向LSTM
        c0 = torch.zeros(self.num_layers * 2, X.shape[0], self.hidden_size).to(self.device)
        
        X = self.embedding(X)
        output, (hn, cn) = self.lstm(X, (h0, c0))

        # 使用最后一个时间步的输出进行分类
        output_pooled = torch.max(output, dim=1)[0]  # 这里使用最大池化
        return self.ff(output_pooled)


In [37]:


lr = 0.1
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda:0')
net = LSTM(num_embeddings=len(vocab),embedding_dim=256,hidden_size=256,device=device)
optimizer = torch.optim.SGD(net.parameters(),lr)


c:\Users\ASUS\.conda\envs\dl\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


In [38]:
data_train[0][0].type()

'torch.LongTensor'

In [39]:
for X,y in train_iter:
    print(X.type())
    break

torch.LongTensor


In [40]:
tools.train(net,train_iter,device,optimizer,criterion)

 16%|█▌        | 101/625 [00:08<00:45, 11.64it/s]

batch100,loss = 0.7932372689247131


 32%|███▏      | 201/625 [00:17<00:36, 11.76it/s]

batch200,loss = 0.6256793141365051


 48%|████▊     | 301/625 [00:25<00:27, 11.59it/s]

...

 80%|████████  | 501/625 [00:42<00:10, 11.92it/s]

batch500,loss = 0.5448476076126099


 96%|█████████▌| 601/625 [00:51<00:02, 11.46it/s]

batch600,loss = 0.4031050503253937


100%|██████████| 625/625 [00:53<00:00, 11.74it/s]


In [41]:
tools.test(net,test_iter,device)

100%|██████████| 157/157 [00:04<00:00, 32.16it/s]

accuracy = 0.8222000002861023
